In [ ]:
import gym
from gym import wrappers
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from collections import defaultdict
import itertools

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = []
        self.previous_board = []

    def reset(self):
      self.board = []
      for i in range(3):
          row = []
          for j in range(3):
              row.append(-1)
          self.board.append(row)
      return self.board

    def step(self, row, col, player):
      if self.board[row][col] == -1:
        self.previous_board = self.board
        self.board[row][col] = player
        # done and this player won, +1 reward
        if self.done(player):
          return self.board, 1, True
        # done and its a draw, 0 reward
        elif self.is_board_filled():
          return self.board, 0, True
        # not done, 0 reward
        else: return self.board, 0, False
      # we tried to change a place on the board that is already filled
      else: return self.board, -5, False

    def done(self, player):
        win = None

        n = len(self.board)

        # checking rows
        for i in range(n):
            win = True
            for j in range(n):
                if self.board[i][j] != player:
                    win = False
                    break
            if win:
                return win

        # checking columns
        for i in range(n):
            win = True
            for j in range(n):
                if self.board[j][i] != player:
                    win = False
                    break
            if win:
                return win

        # checking diagonals
        win = True
        for i in range(n):
            if self.board[i][i] != player:
                win = False
                break
        if win:
            return win

        win = True
        for i in range(n):
            if self.board[i][n - 1 - i] != player:
                win = False
                break
        if win:
            return win
        return False

        for row in self.board:
            for item in row:
                if item == -1:
                    return False
        return True

    # we check if the other player has won (which means that this one lost)
    def player_lost(self, player):
      player = 1 if player == 0 else 1
      return self.done(player)

    def is_board_filled(self):
        for row in self.board:
            for item in row:
                if item == -1:
                    return False
        return True

    def get_possible_moves(self, state):
        moves = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == -1:
                    moves.append([i, j])
        return moves
    
    def undo(self):
        self.board = self.previous_board

    def one_index_to_2D(self, index):
        indices = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
        return indices[index]

    def move_to_index(self, move):
        indices = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
        return indices.index(move)

    def render(self):
        for row in self.board:
            for item in row:
                print(item, end=" ")
            print()
    
    def hash(self, state):
        hash_value = 0
        for i in self.board:
          for j in i:
            if j == -1: hash_value = self.append_digit(3, hash_value)
            elif j == 0: hash_value = self.append_digit(2, hash_value)
            else: hash_value = self.append_digit(1, hash_value)
        return hash_value
    
    def append_digit(self, digit, num): 
      return int(str(digit) + str(num))  

In [ ]:
class N_Step_Q_Learning():
  def __init__(self, env, N, epsilon, learning_rate=0.9, discount_factor=0.95, q_init=0.5):
        self.env = env
        self.player = 0
        self.Q = {}  # [string, np.ndarray]
        self.move_history = []
        self.discount_factor = discount_factor
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.N = N
        super().__init__()

  """
    We get the q-values of all the actions given this state
    If the state is not in the Q-table, we initialize all the q-values as 0.5
  """
  def get_q_value(self, state):
      hashed_state = str(state)
      if hashed_state in self.Q:
        return self.Q[hashed_state]
      else: 
        q_values = np.full(9, 0.5)
        self.Q[hashed_state] = q_values
        return q_values

  """
    We get the move that has the best q-value
  """
  def get_move(self, state):
      hashed_state = str(state)
      q_values = self.get_q_value(state)
      while True:
        move = np.argmax(q_values)
        move = env.one_index_to_2D(move)
        if move in self.env.get_possible_moves(state):
          return move
        else: q_values[self.env.move_to_index(move)] = -1.0

  """
    Randomly select a move from the list of possible moves, or get a move based on learned policy depending on our sample
  """
  def random_epsilon_greedy_policy(self, state):
    sample = random.random()
    if sample > self.epsilon:
      return self.get_move(state)
    else:
      moves = env.get_possible_moves(state)
      hashed_state = str(state)
      if hashed_state in self.Q:
        return moves[random.randrange(len(moves))]
      else: 
        q_values = np.full(9, 0.5)
        self.Q[hashed_state] = q_values
        return moves[random.randrange(len(moves))]

  def train(self, state):
    while True:
      if self.player == 0:
        move = self.random_epsilon_greedy_policy(state)
        self.move_history.append((str(state), self.env.move_to_index(move)))
      else: move = self.make_best_move(state, self.player)
      next_state, reward, done = self.env.step(move[0], move[1], self.player)

      self.player = 0 if self.player == 1 else 1
      state = next_state

      if done:
        num_steps = 0
        for states, moves in self.move_history:
          if num_steps < self.N:
            self.Q[states][moves] += 0#self.learning_rate * (reward + self.discount_factor) * np.max(self.Q[str(next_state)][:] - self.Q[str(states)][moves])
            num_steps += 1
          else: self.Q[states][moves] += 0
        self.env.render()
        print()
        break
  
  """
    This is the expert policy.
    We use the minmax algorithm to get teh score of each move and return the move with max score
  """
  def make_best_move(self, state, player):
    bestScore = -math.inf
    bestMove = None
    env_clone = copy.deepcopy(self.env)
    for move in env_clone.get_possible_moves(state):
        env_clone.step(move[0], move[1], player)
        score = self.minimax(False, player, env_clone)
        env_clone.undo()
        if (score > bestScore):
            bestScore = score
            bestMove = move
    return bestMove

  """
    In short this algorithm makes a search tree for the given state and returns a score
  """
  def minimax(self, isMaxTurn, player, env):
      state = env.board
      # draw
      if (env.is_board_filled()):
          return 0
      # this player won
      elif (env.done(player)):
          return 1
      # this player lost
      elif (env.player_lost(player)): 
          return -1

      scores = []
      for move in env.get_possible_moves(state):
          env.step(move[0], move[1], player)
          scores.append(self.minimax(not isMaxTurn, player, env))
          env.undo()

      return max(scores) if isMaxTurn else min(scores)

env = TicTacToe()
agent = N_Step_Q_Learning(env, N=1, epsilon=0.1)

for e in range(10):
  state = env.reset()
  agent.train(state)

0 1 0 
1 0 1 
0 -1 -1 

1 0 1 
0 1 0 
1 -1 -1 

0 1 1 
0 0 1 
0 -1 -1 

1 0 1 
0 1 0 
1 -1 -1 

0 1 0 
1 0 1 
0 -1 -1 

1 1 0 
0 1 0 
1 0 1 

0 1 1 
0 -1 -1 
0 -1 -1 

1 0 1 
0 1 0 
1 -1 -1 

0 1 0 
1 0 1 
0 -1 -1 

1 0 1 
0 1 0 
1 -1 -1 

